# MODEL EVALUATION WITH CATALAN HELLASWAG

In [4]:
from tqdm import tqdm
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("baiges/CatGPT")
model = AutoModelForCausalLM.from_pretrained("baiges/CatGPT")

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the Hellaswag dataset translated to Catalan
# The dataset is in "pauhidalgoo/hellaswag-val-ca"
dataset = datasets.load_dataset("pauhidalgoo/hellaswag-val-ca")['train']

# Define the evaluation function
def evaluate_model(model, tokenizer, dataset, device):
    total_correct = 0
    total_items = 0

    # Iterate over each entry in the dataset
    for i, item in tqdm(enumerate(dataset)):
        try:
            context = item['ctx']  # Get the context
            endings = item['endings']  # Get the possible endings
            correct_ending_idx = int(item['label'])  # Index of the correct ending

            # Tokenize the context and move it to the device
            context_input = tokenizer(context, return_tensors="pt").input_ids.to(device).long()

            avg_loss_values = []
            for ending in endings:
                try:
                    # Tokenize the ending and move it to the device
                    ending_input = tokenizer(ending, return_tensors="pt").input_ids.to(device).long()
                    
                    # Concatenate the context with each ending
                    input_ids = torch.cat([context_input, ending_input[:, 1:]], dim=-1).to(device).long()

                    # Calculate the loss only over the ending part
                    with torch.no_grad():
                        outputs = model(input_ids)
                        logits = outputs.logits[:, -ending_input.size(1):, :]  # Get logits corresponding to the ending

                        shift_logits = logits[:, :-1, :].contiguous()  # Shift logits to the right
                        shift_labels = ending_input[:, 1:].contiguous()  # Shift labels to the left

                        # Ensure the logits and labels have the same shape
                        shift_logits = shift_logits.view(-1, shift_logits.size(-1))
                        shift_labels = shift_labels.view(-1)

                        # Calculate loss
                        loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
                        loss = loss_fct(shift_logits, shift_labels)
                        avg_loss = loss.mean().item()  # Calculate the average loss per token
                        avg_loss_values.append(avg_loss)
                except RuntimeError as e:
                    print(f"Error processing ending: {e}")
                    avg_loss_values.append(float('inf'))  # Consider high loss if there's an error
                    continue

            # Find the ending with the lowest average loss per token
            predicted_idx = torch.argmin(torch.tensor(avg_loss_values)).item()

            # Check if the prediction is correct and update the counters
            is_correct = predicted_idx == correct_ending_idx
            if is_correct:
                total_correct += 1
            total_items += 1

            # Calculate the accuracy for this specific item
            item_accuracy = total_correct / total_items

            # Print context, prediction, correctness, and accuracy for this item
            print(f"Context: {context}")
            print(f"Predicted Ending: {endings[predicted_idx]}")
            print(f"Correct Ending: {endings[correct_ending_idx]}")
            print(f"Is Correct: {'Yes' if is_correct else 'No'}")
            print(f"Accuracy so far: {item_accuracy:.4f}\n")

        except Exception as e:
            print(f"Error processing item {i}: {e}")
            continue

    # Calculate the overall accuracy
    accuracy = total_correct / total_items
    return accuracy

# Perform the evaluation
accuracy = evaluate_model(model, tokenizer, dataset, device)
print(f"Final Accuracy: {accuracy:.4f}")

Context: [capçalera] Com obtenir la certificació de RCP [títol] Identificar els motius per obtenir la certificació. [pas] Hi ha moltes raons per obtenir la certificació de RCP. Algunes de les més importants inclouen: [subpassos] Pots salvar vides: t'ensenya a ser conscient del teu entorn i a reconèixer els signes d'aturada cardíaca i altres situacions debilitants.
Predicted Ending: Estàs millor equipat per ajudar els altres en necessitat. Els estudis han demostrat que les persones amb certificació de RCP són més propenses a ajudar en cas de necessitat.
Correct Ending: Estàs millor equipat per ajudar els altres en necessitat. Els estudis han demostrat que les persones amb certificació de RCP són més propenses a ajudar en cas de necessitat.
Is Correct: Yes
Accuracy so far: 0.2956



9727it [55:27,  2.26it/s]

Context: [capçalera] Com escriure un perfil corporatiu [títol] Mantingueu el vostre perfil corporatiu concis. [pas] Feu que el vostre perfil corporatiu sigui interessant i fàcil de llegir mantenint-lo tan simple i breu com pugueu. Molts lectors simplement escanejaran el document, digerint frases i conceptes clau.
Predicted Ending: La informació rellevant per a la vostra empresa s'ha d'indicar amb la màxima claredat i concisió possibles. [subpassos] La secció d'operacions i gestió del vostre perfil corporatiu ha d'incloure la màxima quantitat de dades possible sobre la vostra empresa i la vostra indústria en particular.
Correct Ending: Els perfils corporatius poden tenir només uns quants paràgrafs o més de 30 pàgines de llarg, però penseu en quina informació és realment important abans d'escollir un perfil de llarg format. [subpassos] Si esteu creant el vostre perfil corporatiu en línia, manteniu-lo curt enllaçant a informació més detallada en pàgines separades. 
Is Correct: No
Accuracy

9728it [55:27,  2.32it/s]

Context: [capçalera] Com fer art d'ungles amb flors [títol] Elimina qualsevol esmalt esmicolat que quedi a les teves ungles. [pas] Abans de començar a pintar, vols donar-te un bell llenç sobre el qual treballar. Pots comprar dissolvent d'esmalt a la teva farmàcia local.
Predicted Ending: [subpassos] El dissolvent per a art d'ungles no és abrasiu i no irritarà la pell. No et rentes les mans abans de començar a pintar per evitar que l'esmalt es transfereixi a les teves ungles.
Correct Ending: Si les teves ungles són propenses a assecar-se i descamar-se, busca un dissolvent etiquetat com "sense acetona". Aquests són més suaus amb les teves ungles, i no vols assecar-les just abans de fer-te una manicura! [títol] Talla i lima les teves ungles.
Is Correct: No
Accuracy so far: 0.2956



...

Context: [capçalera] Com calcular quant diners necessites per jubilar-te [títol] Determina les despeses bàsiques de la vida. [pas] Un primer pas important és determinar la quantitat necessària per cobrir les despeses bàsiques de la vida cada any. Hi ha diferents perspectives sobre quant serà.
Predicted Ending: Algunes fonts suggereixen fins a cinc vegades el cost del mercat per any. Altres suggereixen més de vuit vegades el cost real per a la vida bàsica.
Correct Ending: [subpassos] Alguns experts creuen que simplement hauries de calcular les teves despeses actuals. Després, espera que necessitaràs aproximadament la mateixa quantitat per viure una vegada que et jubilis.
Is Correct: No
Accuracy so far: 0.2947



10041it [57:13,  2.99it/s]

Context: [capçalera] Com consolar una amiga adolescent embarassada [títol] Escolta i ofereix suport amb els braços oberts. [pas] Fes saber a la teva amiga que estàs disposada a escoltar-la si necessita parlar. No obstant això, si la teva amiga diu que necessita una mica de temps per processar les coses per si mateixa primer, no la pressionis a parlar.
Predicted Ending: Dona'ls temps i simplement recorda-li que estàs allà per a ella si vol parlar. [subpassos] Intenta dir alguna cosa com: 'Sé que estàs sobrecarregada ara mateix, però estic aquí per a tu i estic disposada a escoltar-te si mai necessites parlar'.
Correct Ending: Dona'ls temps i simplement recorda-li que estàs allà per a ella si vol parlar. [subpassos] Intenta dir alguna cosa com: 'Sé que estàs sobrecarregada ara mateix, però estic aquí per a tu i estic disposada a escoltar-te si mai necessites parlar'.
Is Correct: Yes
Accuracy so far: 0.2947



10042it [57:14,  2.92it/s]

Context: [capçalera] Com dormir millor en un hotel [títol] Porta taps per les orelles. [pas] Comença empacant taps per les orelles per a la teva estada a l'hotel per ajudar a bloquejar qualsevol soroll. Potser tens un parell de taps per les orelles de viatge que acostumes a portar amb tu en els viatges.
Predicted Ending: O pots comprar un parell de taps per les orelles per portar-los amb tu per poder dormir millor a l'hotel, malgrat qualsevol soroll. Empaca els teus taps per les orelles perquè tinguis una manera de bloquejar el soroll a la nit.
Correct Ending: O pots comprar un parell de taps per les orelles per portar-los amb tu per poder dormir millor a l'hotel, malgrat qualsevol soroll. Empaca els teus taps per les orelles perquè tinguis una manera de bloquejar el soroll a la nit.
Is Correct: Yes
Accuracy so far: 0.2948

Final Accuracy: 0.2948
